# --- STEP 1: IMPORT & EXTRACT ---

In [1]:
import pandas as pd
import re

# Load Dataset

In [2]:
df_participant = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
print(df_participant.head(10))

                         participant_id first_name   last_name   birth_date  \
0  bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a      Citra  Nurdiyanti  05 Feb 1991   
1  7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3       Aris    Setiawan  11 Jan 1993   
2  19582d7f-b824-4fe5-a517-d5bf573fc768   Cornelia   Handayani  31 Jul 1993   
3  aeb6d817-25f3-4867-8a74-8d92e0a0f633      Soleh      Rajasa  04 Nov 1991   
4  1fdabdd9-5444-4c97-87b2-fe8833ad0d27       Vivi      Astuti  22 Jan 2003   
5  f1e8749d-6dec-4ede-be25-efb31ce80f99  Bahuraksa  Mangunsong  13 Nov 1997   
6  707219fa-cb0b-467c-ad32-e99c61603020      Tiara     Maryati  10 Mar 1994   
7  bc4e9cc3-f49b-40a2-905c-3e9dcf3af958      Dimaz     Gunarto  16 Feb 2000   
8  7ca0a2e8-bab0-42a3-b6aa-ef4df35c7800       Zizi   Puspasari  11 Apr 2002   
9  7db879a6-eae5-4cf3-87da-c452df8e99c1      Cecep    Haryanto  15 Jan 1994   

                                             address        phone_number  \
0           Gg. Monginsidi No. 08\nMedan, Aceh 80734  

# --- STEP 2: TRANSFORMATION (CLEANING & ENGINEERING) ---

In [5]:
# 1. Postal Code (Ambil angka di akhir alamat)
df_participant['postal_code'] = df_participant['address'].str.extract(r'(\d+)$')

In [6]:
# 2. City (Ambil kata setelah newline sebelum koma)
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)')

In [7]:
# 3. Github Profile (Gabungin link + nama lowercase)
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()

In [8]:
# 4. Phone Number Cleaning (Hapus +62, kurung, spasi)
# Tambahkan regex=True agar pattern dibaca sebagai Regex
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0', regex=True)
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '', regex=True)
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '', regex=True)

In [9]:
# 5. Team Name (Singkatan Nama + Negara + Singkatan Institusi)
def generate_team_name(col):
    abbrev_name = "%s%s"%(col['first_name'][0],col['last_name'][0])
    country = col['country']
    abbrev_institute = '%s'%(''.join(list(map(lambda word: word[0], col['institute'].split()))))
    return "%s-%s-%s"%(abbrev_name,country,abbrev_institute)

df_participant['team_name'] = df_participant.apply(generate_team_name, axis=1)

In [10]:
# 6. Email Generation (Logic Universitas vs Umum)
def generate_email(col):
    first_name_lower = col['first_name'].lower()
    last_name_lower = col['last_name'].lower()
    institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split())))

    if 'Universitas' in col['institute']:
        if len(col['country'].split()) > 1:
            country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
        else:
            country = col['country'][:3].lower()
        return "%s%s@%s.ac.%s"%(first_name_lower,last_name_lower,institute,country)

    return "%s%s@%s.com"%(first_name_lower,last_name_lower,institute)

df_participant['email'] = df_participant.apply(generate_email, axis=1)

In [11]:
# 7. Birth Date Formatting (ke YYYY-MM-DD)
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')

In [12]:
# 8. Register Time Formatting (Unix ke Datetime)
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')

# --- STEP 3: FINAL OUTPUT ---

In [13]:
# Tampilkan data final
pd.set_option('display.max_columns', None)
print("Transformation Complete! Here is the final schema:")
print(df_participant[['first_name', 'last_name', 'email', 'cleaned_phone_number', 'team_name', 'register_at']].head(10))

Transformation Complete! Here is the final schema:
  first_name   last_name                        email cleaned_phone_number  \
0      Citra  Nurdiyanti      citranurdiyanti@upm.com          01510812706   
1       Aris    Setiawan        arissetiawan@ud.ac.ku          00364617027   
2   Cornelia   Handayani    corneliahandayani@uhp.com           0898336695   
3      Soleh      Rajasa          solehrajasa@pty.com          04183294756   
4       Vivi      Astuti           viviastuti@phr.com           0812511835   
5  Bahuraksa  Mangunsong  bahuraksamangunsong@cmh.com           0113697082   
6      Tiara     Maryati         tiaramaryati@uiw.com          02444307220   
7      Dimaz     Gunarto          dimazgunarto@pp.com          00569319059   
8       Zizi   Puspasari     zizipuspasari@ubs.ac.gam           0118261289   
9      Cecep    Haryanto       cecepharyanto@pppt.com           0861561327   

                         team_name         register_at  
0                   CN-Georgia-UP